In [1]:
# from __future__ import print_function
from numpy.random import seed

# TensorFlow and tf.keras
import tensorflow as tf
import tensorflow.keras as keras

from DataGenerator import DataGenerator
from i3d_inception import *

# Helper libraries
seed(4)
import numpy as np
import matplotlib

matplotlib.use('Agg')
from matplotlib import pyplot as plt
import os
import h5py
import scipy.io as sio
import glob
import gc

from keras.models import load_model, Model, Sequential
from keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Activation, Dense, Dropout, ZeroPadding2D)
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix, accuracy_score, auc, roc_curve
from sklearn.model_selection import KFold, StratifiedShuffleSplit
from scipy import interp
from keras.layers.advanced_activations import ELU

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import keras.backend as K

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session


Using TensorFlow backend.

Bad key "text.kerning_factor" on line 4 in
c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# CHANGE THESE VARIABLES ---
data_folder = r'C:\Users\kentw\OneDrive - University of Toronto\PycharmProjects\Fall-Detection-with-CNNs-and-Optical-Flow\MAA' 
#mean_file = 'flow_mean.mat'  # Used as a normalisation for the input of the network

save_features = False  # Boolean flag if we save features in h5py
save_plots = True
load_data = False

# Set to 'True' if you want to restore a previous trained models
# Training is skipped and test is done
use_checkpoint = True # Set to True or False
# --------------------------

best_model_path = 'models/'
plots_folder = 'plots/'

features_file = 'features_MAA.h5'
labels_file = 'labels_MAA.h5'
features_key = 'features'
labels_key = 'labels'

rgb_file = 'rgb_MAA.h5'
rgb_key = 'rgb'

# Hyper parameters
batch_norm = False
gpu_num = 1 
num_features = [None, 7, 1, 1, 1024]  # Specific dimension of features for 64-frame clips
learning_rate = 0.01
mini_batch_size = 1
weight_0 = 1 # higher weight of 0 prioritizes learning in class 0
epochs = 60
dropout_prob=0.36

optimizer = 'adam'
weights = "rgb_imagenet_and_kinetics"
name = 'train_nn'

# Name of the experiment
exp = '_lr{}_batchs{}_batchnorm{}_w0_{}_{}_{}'.format(learning_rate,
                                                      mini_batch_size,
                                                      batch_norm,
                                                      weight_0, 
                                                      name,
                                                      weights)
# Input dimensions
NUM_FRAMES = 64
FRAME_HEIGHT = 224
FRAME_WIDTH = 224
NUM_RGB_CHANNELS = 3
NUM_CLASSES = 2

In [3]:
# Functions 
def plot_training_info(case, metrics, save, history):
    """
    Function to create plots for train and validation loss and accuracy
    Input:
    * case: name for the plot, an 'accuracy.png' or 'loss.png' will be concatenated after the name.
    * metrics: list of metrics to store: 'loss' and/or 'accuracy'
    * save: boolean to store the plots or only show them.
    * history: History object returned by the Keras fit function.
    """
    plt.ioff()
    if 'accuracy' in metrics:
        fig = plt.figure()
        plt.plot(history['acc'])
        plt.plot(history['val_acc'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'val'], loc='upper left')
        if save:
            plt.savefig(case + 'accuracy.png')
            plt.gcf().clear()
        else:
            plt.show()
        plt.close(fig)

    # summarize history for loss
    if 'loss' in metrics:
        fig = plt.figure()
        plt.plot(history['loss'])
        plt.plot(history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        # plt.ylim(1e-3, 1e-2)
        plt.yscale("log")
        plt.legend(['train', 'val'], loc='upper left')
        if save:
            plt.savefig(case + 'loss.png')
            plt.gcf().clear()
        else:
            plt.show()
        plt.close(fig)


def list_data(exp_name):
    """
    Function to list existing data set by filename and label 
    """
    vids = {'pass': [], 'fail': []}
    global data_folder
    data_folder = [os.path.join(data_folder, category) for category in experiments[exp_name]]
    for path in data_folder:
        for file in os.listdir(path):
            if not file.lower().endswith('.npy'):
                continue
            else:
                file_dir = os.path.join(path, file)
                ID = file_dir.split('\\')[-1].rstrip('.npy')
                # Classify
                result = ID.split('_')[2][1]
                if result == "P": vids["pass"].append(ID)
                else: vids["fail"].append(ID)
    return vids


def saveFeatures(feature_extractor,
                 features_file,
                 labels_file,
                 features_key,
                 labels_key):
    """
    Function to load the RGB data, do a feed-forward through the feature extractor and store the
    output feature vectors in the file 'features_file' and the  labels in 'labels_file'.
    Input:
    * feature_extractor: model without the top layer, which is the classifer
    * features_file: path to the hdf5 file where the extracted features are going to be stored
    * labels_file: path to the hdf5 file where the labels of the features are going to be stored
    * features_key: name of the key for the hdf5 file to store the features
    * labels_key: name of the key for the hdf5 file to store the labels
    """

    # Fill the IDs and classes arrays
    classes = list_data(exp_name)
    class0, class1 = classes['fail'], classes['pass']
    num_samples = len(class0) + len(class1)
    num_features[0] = num_samples
    
    # File to store the extracted features and datasets to store them
    # IMPORTANT NOTE: 'w' mode totally erases previous data
    # Write files in h5py format
    h5features = h5py.File(features_file, 'w')
    h5labels = h5py.File(labels_file, 'w')
    
    # Create data sets
    dataset_features = h5features.create_dataset(features_key,
                                                 shape=num_features,
                                                 dtype='float64')
    dataset_labels = h5labels.create_dataset(labels_key,
                                             shape=(num_samples, 1),
                                             dtype='float64')

    # Process class 0 
    gen = DataGenerator(class0, np.zeros(len(class0)), 1)
    for i in range(len(class0)):
                        
        rgb_images, rgb_labels = gen.__getitem__(i)
        predictions = feature_extractor.predict(rgb_images)

        dataset_features[i, :] = predictions
        dataset_labels[i, :] = 0
                
        del rgb_images, rgb_labels
        gc.collect()
        
    # Process class 1
    gen = DataGenerator(class1, np.ones(len(class1)), 1)
    for i in range(len(class0), num_samples):
        
        rgb_images, rgb_labels = gen.__getitem__(i)        
        prediction = feature_extractor.predict(rgb_images)

        dataset_features[i, :] = predictions
        dataset_labels[i, :]= 1

        del rgb_images, rgb_labels
        gc.collect()

    h5features.close()
    h5labels.close()


# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del model # this is from global space - change this as you need
        del x
    except:
        pass

    # print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.99
    config.gpu_options.allow_growth = True
    K.set_session(tf.Session(config=config))


def build_model():
    # I3D ARCHITECTURE
    model = Inception_Inflated3d(
        include_top=False ,
        weights=weights,
        input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_RGB_CHANNELS),
        classes=NUM_CLASSES)

    # ==================== CLASSIFIER ========================
    # Batch size of 1 does not need normalization
    # if batch_norm:
      #   x = BatchNormalization(axis=-1, momentum=0.99,
        #                       epsilon=0.001)(model.output)
        
    x = Dropout(dropout_prob)(model.output)

    x = conv3d_bn(x, NUM_CLASSES, 1, 1, 1, padding='same', 
            use_bias=True, use_activation_fn=False, use_bn=False, name='Conv3d_6a_1x1')

    num_frames_remaining = int(x.shape[1])
    x = Reshape((num_frames_remaining, NUM_CLASSES))(x)

    # logits (raw scores for each class)
    x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
               output_shape=lambda s: (s[0], s[2]))(x)

    # if not endpoint_logit
    x = Activation('softmax', name='prediction')(x)

    #compile the model
    return Model(input = model.input, output = x)


In [4]:
# trainer 
def main(exp_name):
    global exp
    exp = exp_name + exp
    
    do_training = True
    compute_metrics = True
    threshold = 0.5

    if do_training:
        # Import data
        if load_data:
            load_data(rgb_file,
                      labels_file,
                      rgb_key,
                      labels_key)
            
        # # Import data
        # h5rgb = h5py.File(rgb_file, 'r')
        # h5labels = h5py.File(labels_file, 'r')
        
        classes = list_data(exp_name)
        class0, class1 = classes['fail'], classes['pass']
        X_full = np.asarray((class0 + class1))
        X_full = np.expand_dims(X_full, axis=1)
        _y_full = np.concatenate(((np.zeros(shape = (len(class0),1))), np.ones(shape = (len(class1),1))))
        
        # X_full will contain all the feature vectors extracted
        # X_full = h5rgb[rgb_key]
        # _y_full = np.asarray(h5labels[labels_key])
        # print(X_full, _y_full)
        
        # Indices of 0 and 1 in the data set
        zeroes_full = np.asarray(np.where(_y_full == 0)[0])
        ones_full = np.asarray(np.where(_y_full == 1)[0])
        zeroes_full.sort()
        ones_full.sort()

        # Traditional Machine Learning methodology
        # Method get_n_splits() returns the number of splits
        
        kf_0 = KFold(n_splits=5, shuffle=True)
        kf_0.get_n_splits(X_full[zeroes_full, ...])

        kf_1 = KFold(n_splits=5, shuffle=True)
        kf_1.get_n_splits(X_full[ones_full, ...])

        sensitivities = []
        specificities = []
        fars = []
        mdrs = []
        accuracies = []
        f1s = []
        tps = []
        tns = []
        fps = []
        fns = []

        fold_number = 1 
        
        tprs = []
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)
        fig, ax = plt.subplots() # For ROC
        
        # CROSS-VALIDATION: Stratified partition of the dataset into train/test sets
        for ((train_index_0, test_index_0),
             (train_index_1, test_index_1)) in zip(
            kf_0.split(X_full[zeroes_full, ...]),
            kf_1.split(X_full[ones_full, ...])
        ):
            
            train_index_0 = np.asarray(train_index_0)
            test_index_0 = np.asarray(test_index_0)
            train_index_1 = np.asarray(train_index_1)
            test_index_1 = np.asarray(test_index_1)

            # Train and Test Set
            X = np.concatenate((X_full[zeroes_full, ...][train_index_0, ...],
                                X_full[ones_full, ...][train_index_1, ...]))
            _y = np.concatenate((_y_full[zeroes_full, ...][train_index_0, ...],
                                 _y_full[ones_full, ...][train_index_1, ...]))
            X2 = np.concatenate((X_full[zeroes_full, ...][test_index_0, ...],
                                 X_full[ones_full, ...][test_index_1, ...]))
            _y2 = np.concatenate((_y_full[zeroes_full, ...][test_index_0, ...],
                                  _y_full[ones_full, ...][test_index_1, ...]))

            # Create a validation subset from the training set
            val_size = 0.5
            
            zeroes = np.asarray(np.where(_y == 0)[0])
            ones = np.asarray(np.where(_y == 1)[0])
            
            zeroes.sort()
            ones.sort()

            trainval_split_0 = StratifiedShuffleSplit(n_splits=1,
                                                      test_size=val_size / 2,
                                                      random_state=1)
            indices_0 = trainval_split_0.split(X[zeroes, ...],
                                               np.argmax(_y[zeroes, ...], 1))
            trainval_split_1 = StratifiedShuffleSplit(n_splits=1,
                                                      test_size=val_size / 2,
                                                      random_state=1)
            indices_1 = trainval_split_1.split(X[ones, ...],
                                               np.argmax(_y[ones, ...], 1))
            
            train_indices_0, val_indices_0 = indices_0.__next__()
            train_indices_1, val_indices_1 = indices_1.__next__()

            X_train = np.concatenate([X[zeroes, ...][train_indices_0, ...],
                                      X[ones, ...][train_indices_1, ...]], axis=0)
            y_train = np.concatenate([_y[zeroes, ...][train_indices_0, ...],
                                      _y[ones, ...][train_indices_1, ...]], axis=0)
            X_val = np.concatenate([X[zeroes, ...][val_indices_0, ...],
                                    X[ones, ...][val_indices_1, ...]], axis=0)
            y_val = np.concatenate([_y[zeroes, ...][val_indices_0, ...],
                                    _y[ones, ...][val_indices_1, ...]], axis=0)
            
            X_train = X_train.flatten()
            X_val = X_val.flatten()
            y_train = y_train.flatten()
            y_val = y_val.flatten()
            X2 = X2.flatten()
            _y2 = _y2.flatten()
            
            # Generators
            training_generator = DataGenerator(X_train, y_train, mini_batch_size*gpu_num)
            validation_generator = DataGenerator(X_val, y_val, mini_batch_size*gpu_num)

            reset_keras()
    
            model = build_model()
            adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
            # opt = SGD(lr=learning_rate, momentum=0.0, nesterov=False)
            model.compile(adam, loss='binary_crossentropy', metrics=['accuracy'])
            
            fold_best_model_path = best_model_path + exp + '_MAA_fold_{}.h5'.format(fold_number)
            
            if not use_checkpoint:
                # ==================== TRAINING ========================
                # weighting of each class: only the fall class gets
                # a different weight
                class_weight = {0: weight_0, 1: 1}

                # callback definition
                # Choose the monitored metric
                metric = 'val_loss'
                e = EarlyStopping(monitor=metric, min_delta=0, patience=5,
                                  mode='auto', restore_best_weights=True)
                c = ModelCheckpoint(fold_best_model_path,
                                    monitor=metric,
                                    save_best_only=True,
                                    save_weights_only=False, mode='auto')
                callbacks = [e, c]

                # Batch training
                history = model.fit_generator(generator=training_generator,
                                         validation_data=validation_generator,
                                         steps_per_epoch=len(training_generator),
                                         nb_epoch=epochs,
                                         class_weight=class_weight,
                                         callbacks=callbacks, 
                                         use_multiprocessing=True,
                                         workers=6,
                                         shuffle=True)
               
                plot_training_info(plots_folder + exp + '_fold{}'.format(fold_number), ['accuracy', 'loss'],
                                   save_plots, history.history)
                
            # ==================== EVALUATION ========================
            
            # Load best model
            print('\nModel loaded from checkpoint')

            model = load_model(fold_best_model_path)

            test_generator = DataGenerator(X2, _y2, shuffle=False)
            
            if compute_metrics:
                predicted = model.predict_generator(test_generator)
                y_score = np.copy(predicted)

                predicted = predicted[:,1]

                for i in range(len(predicted)):
                    if predicted[i] < threshold:
                        predicted[i] = 0
                    else:
                        predicted[i] = 1

                # Array of predictions 0/1
                predicted = np.asarray(predicted).astype(int)
                
                # Compute metrics and print them
                cm = confusion_matrix(_y2, predicted, labels=[0, 1])
                tp = cm[0][0]
                fn = cm[0][1]
                fp = cm[1][0]
                tn = cm[1][1]
                
                tpr = tp / float(tp + fn)
                fpr = fp / float(fp + tn)
                fnr = fn / float(fn + tp)
                tnr = tn / float(tn + fp)
                accuracy = accuracy_score(_y2, predicted)
                precision = tp / float(tp + fp)
                recall = tp / float(tp + fn)
                specificity = tn / float(tn + fp)
        
                try:
                    f1 = 2 * float(precision * recall) / float(precision + recall)                
                except:
                    f1 = 0
                    print("An exception occurred")
                
                print('\n')
                print('FOLD {} results:'.format(fold_number))
                print('TP: {}, TN: {}, FP: {}, FN: {}'.format(tp, tn, fp, fn))
                print('TPR: {}, TNR: {}, FPR: {}, FNR: {}'.format(tpr, tnr, fpr, fnr))
                print('Sensitivity/Recall: {}'.format(recall))
                print('Specificity: {}'.format(specificity))
                print('Precision: {}'.format(precision))
                print('F1-measure: {}'.format(f1))
                print('Accuracy: {}'.format(accuracy))
                print('\n')

                # Store the metrics for this epoch
                sensitivities.append(tp / float(tp + fn))
                specificities.append(tn / float(tn + fp))
                fars.append(fpr)
                mdrs.append(fnr)
                accuracies.append(accuracy)
                f1s.append(f1)
                
                tps.append(tp)
                tns.append(tn)
                fps.append(fp)
                fns.append(fn)
                
                # Binarize the output
                _y2 = label_binarize(_y2, classes=[1, 0])
                n_classes = _y2.shape[1] 
                
                fpr = dict()
                tpr = dict()
                roc_auc = dict()
                
                for i in range(n_classes):
                    fpr[i], tpr[i], _ = roc_curve(_y2[:, i], y_score[:, i])
                    roc_auc[i] = auc(fpr[i], tpr[i])
    
                lw = 2
                color = ['g', 'r', 'c', 'm', 'y']
                plt.plot(fpr[0], tpr[0],
                         lw=lw, color=color[fold_number - 1],
                         label= 'ROC fold {}'.format(fold_number) + ' ' + '(area = %0.2f)' % roc_auc[0])
                plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
                plt.xlim([0.0, 1.0])
                plt.ylim([0.0, 1.05])
                plt.xlabel('False Positive Rate', fontfamily= 'Palatino Linotype', fontsize=15)
                plt.ylabel('True Positive Rate', fontfamily= 'Palatino Linotype', fontsize=15)
                plt.title('Experiment 3 ROC Curves of Class Slips', fontfamily= 'Palatino Linotype', fontsize=15)
                plt.legend(loc="lower right")
                
                font = {'family' : 'Palatino Linotype', 'size':  10}
                plt.rc('font', **font)
                
                # ROC curve plotting
                interp_tpr = np.interp(mean_fpr, fpr[0], tpr[0])
                interp_tpr[0] = 0.0
                tprs.append(interp_tpr)
                aucs.append(roc_auc[0])
                
                fold_number += 1

                
    print('5-FOLD CROSS-VALIDATION RESULTS ===================')
    print("Sensitivity: %.2f (+/- %.2f)" % (np.mean(sensitivities), np.std(sensitivities)))
    print("Specificity: %.2f (+/- %.2f)" % (np.mean(specificities), np.std(specificities)))
    print("FAR: %.2f (+/- %.2f)" % (np.mean(fars), np.std(fars)))  # False alarm rates 
    print("MDR: %.2f (+/- %.2f)" % (np.mean(mdrs), np.std(mdrs)))  # Missed detection rates
    print("Accuracy: %.2f (+/- %.2f)" % (np.mean(accuracies), np.std(accuracies)))
    print("F1: %.2f (+/- %.2f)" % (np.mean(f1s), np.std(f1s)))
    
    print("tp: %.2f (+/- %.2f)" % (np.mean(tps), np.std(tps)))
    print("fp: %.2f (+/- %.2f)" % (np.mean(fps), np.std(fps)))
    print("tn: %.2f (+/- %.2f)" % (np.mean(tns), np.std(tns)))
    print("fn: %.2f (+/- %.2f)" % (np.mean(fns), np.std(fns)))
 
    # ROC
    font = {'family' : 'Palatino Linotype', 'size':  12}
    plt.rc('font', **font)
    
    ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr, color='b',
            label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
            lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                    label=r'$\pm$ 1 std. dev.')
    
    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],)
    ax.legend(loc="lower right")
    
    plt.savefig('ROC for exp3')
    plt.show()

In [5]:
if __name__ == '__main__':
    if not os.path.exists(best_model_path):
        os.makedirs(best_model_path)
    if not os.path.exists(plots_folder):
        os.makedirs(plots_folder)
        
    # Experiments
    # 1. Only with hazardous slips (with more data you gathered) --> 60:60--> same as last time with more data
    # 2. Only with normal slips and 'no slips' --> 60:60 --> this can be a fair detection 
    # 3. Combination of 1+2 --> 120:120
    # 4. With small slips --> 60:60
    # 5. With all dataset --> 180:180
    
    experiments = {'exp1':['Hazardous_slips', 'Pass_split1'],
                   'exp2':['Normal_slips', 'Pass_split1'],
                   'exp3':['Hazardous_slips', 'Normal_slips', 'Pass_split1', 'Pass_split2'],
                   'exp4':['Small_slips', 'Pass_split1'],
                   'exp5':['Hazardous_slips', 'Normal_slips', 'Pass_split1', 'Pass_split2', 'Small_slips', 'Pass_split3']
                  }
    
    main('exp4') # Make sure you change ROC manually
    
    # Learning rate
    # Memory usage
    # Patience
    # ROC title and name
    # Not use check model
    # main function argument
    # Change seed 










Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Model loaded from checkpoint


c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\ipykernel_launcher.py:185: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`






FOLD 1 results:
TP: 10, TN: 9, FP: 3, FN: 2
TPR: 0.8333333333333334, TNR: 0.75, FPR: 0.25, FNR: 0.16666666666666666
Sensitivity/Recall: 0.8333333333333334
Specificity: 0.75
Precision: 0.7692307692307693
F1-measure: 0.8
Accuracy: 0.7916666666666666




c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\ipykernel_launcher.py:185: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`



Model loaded from checkpoint


FOLD 2 results:
TP: 9, TN: 9, FP: 3, FN: 3
TPR: 0.75, TNR: 0.75, FPR: 0.25, FNR: 0.25
Sensitivity/Recall: 0.75
Specificity: 0.75
Precision: 0.75
F1-measure: 0.75
Accuracy: 0.75




...

c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\ipykernel_launcher.py:185: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`



Model loaded from checkpoint


FOLD 4 results:
TP: 8, TN: 12, FP: 0, FN: 4
TPR: 0.6666666666666666, TNR: 1.0, FPR: 0.0, FNR: 0.3333333333333333
Sensitivity/Recall: 0.6666666666666666
Specificity: 1.0
Precision: 1.0
F1-measure: 0.8
Accuracy: 0.8333333333333334




c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\ipykernel_launcher.py:185: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`



Model loaded from checkpoint


FOLD 5 results:
TP: 7, TN: 10, FP: 2, FN: 5
TPR: 0.5833333333333334, TNR: 0.8333333333333334, FPR: 0.16666666666666666, FNR: 0.4166666666666667
Sensitivity/Recall: 0.5833333333333334
Specificity: 0.8333333333333334
Precision: 0.7777777777777778
F1-measure: 0.6666666666666666
Accuracy: 0.7083333333333334


5-FOLD CROSS-VALIDATION RESULTS ===================
Sensitivity: 0.73 (+/- 0.10)
Specificity: 0.80 (+/- 0.11)
FAR: 0.20 (+/- 0.11)
MDR: 0.27 (+/- 0.10)
Accuracy: 0.77 (+/- 0.04)
F1: 0.76 (+/- 0.05)
tp: 8.80 (+/- 1.17)
fp: 2.40 (+/- 1.36)
tn: 9.60 (+/- 1.36)
fn: 3.20 (+/- 1.17)


c:\users\kentw\appdata\local\continuum\anaconda3\envs\keras-kinetics-i3d\lib\site-packages\ipykernel_launcher.py:315: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
